In [2]:
import re
import xml.etree.ElementTree as ET

# Dictionary parsing

In [3]:
dict_path = r'stuff/dict.opcorpora.xml'

In [4]:
def parse_dict(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    return root

In [5]:
dict_root = parse_dict(dict_path)

# Parsing lemmata from dictionary

In [6]:
from collections import defaultdict

In [7]:
def parse_lemmata(root):
    lemmata_dict = defaultdict(list)
    lemmata_section = root.find('lemmata')

    for lemma_elem in lemmata_section.findall('lemma'):
        l_elem = lemma_elem.find('l')
        lemma_text = l_elem.get('t', '')
        
        first_g_elem = l_elem.find('g')
        main_grammem = first_g_elem.get('v', '') if first_g_elem is not None else ''
                    
        for f_elem in lemma_elem.findall('f'):
            form_text = f_elem.get('t', '')
            if form_text: 
                new_entry = (lemma_text, main_grammem)
                
                if new_entry not in lemmata_dict[form_text]:
                    lemmata_dict[form_text].append(new_entry)
            
    return dict(lemmata_dict)

In [8]:
lemmata_dict = parse_lemmata(dict_root)

In [10]:
lemmata_dict['абдужалия']

[('абдужалия', 'NOUN')]

# Tokenize and lemmatize text

In [9]:
def tokenize(text):  
    tokens = re.findall(r'\w+', text)
    return [token for token in tokens if token]

def normalize(word):
    return word.lower()

In [ ]:
def process_text(input_text, lemmata_dict):
    lines = input_text.strip().split('\n')
    results = []
    
    for line in lines:
        tokens = tokenize(line)
        processed_tokens = []
        
        for token in tokens:
            normalized = normalize(token)
            
            if normalized in lemmata_dict:
                lemma, grammem = lemmata_dict[normalized]
                processed_tokens.append(f"{token}{{{lemma}={grammem}}}")
            else:
                processed_tokens.append(f"{token}{{{normalized}=UNKN}}")
        
        results.append(' '.join(processed_tokens))
    
    return '\n'.join(results)

In [61]:
text = '''Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.
Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.'''

In [62]:
process_text(text, lemmata_dict)

'Стала{стал=VERB} стабильнее{стабильнее=COMP} экономическая{экономический=ADJF} и{и=CONJ} политическая{политический=ADJF} обстановка{обстановка=NOUN} предприятия{предприятие=NOUN} вывели{вывел=VERB} из{из=PREP} тени{теню=VERB} зарплаты{зарплата=NOUN} сотрудников{сотрудник=NOUN}\nВсе{весь=ADJF} Гришины{гришины=NOUN} одноклассники{одноклассник=NOUN} уже{уж=NOUN} побывали{побывал=VERB} за{за=PREP} границей{граница=NOUN} он{он=NPRO} был{есть=VERB} чуть{чуть=ADVB} ли{ли=CONJ} не{не=PRCL} единственным{единственный=ADJF} кого{кто=NPRO} не{не=PRCL} вывозили{вывожу=VERB} никуда{никуда=ADVB} дальше{дальше=COMP} Красной{красная=NOUN} Пахры{пахра=NOUN}'